In [6]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align import AlignInfo
from Bio import AlignIO
from Bio.Align.Applications import MafftCommandline
import tempfile
import os
import subprocess
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
input_gbff = "genomic.gbff"
output_fasta = "ecoli_rRNA_consensus.fasta"

In [ ]:
# Step 1: Collect sequences by subtype
rRNA_seqs = {"16S": [], "23S": [], "5S": []}

for record in SeqIO.parse(input_gbff, "genbank"):
    for feature in record.features:
        if feature.type == "rRNA":
            product = feature.qualifiers.get("product", [""])[0]
            seq = feature.extract(record.seq)
            if "16S" in product:
                rRNA_seqs["16S"].append(SeqRecord(seq, id=product, description=""))
            elif "23S" in product:
                rRNA_seqs["23S"].append(SeqRecord(seq, id=product, description=""))
            elif "5S" in product:
                rRNA_seqs["5S"].append(SeqRecord(seq, id=product, description=""))

def get_consensus(seqs, label):
    with tempfile.TemporaryDirectory() as tmpdir:
        fasta_path = os.path.join(tmpdir, f"{label}.fasta")
        aligned_path = os.path.join(tmpdir, f"{label}_aligned.fasta")

        # Write unaligned sequences to FASTA
        SeqIO.write(seqs, fasta_path, "fasta")

        # Run MAFFT alignment
        mafft_cline = MafftCommandline(input=fasta_path)
        stdout, stderr = mafft_cline()

        # Save aligned output to file (optional but recommended)
        with open(aligned_path, "w") as handle:
            handle.write(stdout)

        # Read aligned sequences
        alignment = AlignIO.read(aligned_path, "fasta")
        summary_align = AlignInfo.SummaryInfo(alignment)
        consensus = summary_align.dumb_consensus(threshold=0.7, ambiguous="N")

        return SeqRecord(consensus, id=f"{label}_consensus", description="")

# Step 3: Write all consensus sequences
consensus_records = []
for rna_type in ["16S", "23S", "5S"]:
    if rRNA_seqs[rna_type]:
        print(f"Generating consensus for {rna_type} rRNA...")
        consensus_seq = get_consensus(rRNA_seqs[rna_type], rna_type)
        consensus_records.append(consensus_seq)

SeqIO.write(consensus_records, output_fasta, "fasta")
print(f"\n✅ Consensus rRNA reference written to: {output_fasta}")

In [ ]:
# Settings
reference_fasta = "ecoli_rRNA_consensus.fasta"
bowtie_index_prefix = "ecoli_rRNAs"  # Bowtie2 will generate files with this prefix

In [ ]:
# Step 1: Build bowtie2 index if not already done
if not all(os.path.exists(f"{bowtie_index_prefix}.{ext}") for ext in ['1.bt2', '2.bt2', '3.bt2', '4.bt2', 'rev.1.bt2', 'rev.2.bt2']):
    print("Building Bowtie2 index...")
    subprocess.run(["bowtie2-build", reference_fasta, bowtie_index_prefix], check=True)
else:
    print("Bowtie2 index already exists. Skipping index build.")

In [ ]:
# Step 2: Align all .fastq.gz files in current directory
fastq_files = sorted(glob("*.fastq.gz"))
fastq_files

In [ ]:
for fq in fastq_files:
    sample_name = fq.replace(".fastq.gz", "")
    bam_unsorted = f"{sample_name}.unsorted.bam"
    bam_sorted = f"{sample_name}.sorted.bam"

    print(f"\nProcessing {fq}...")

    # Align and convert to BAM
    with subprocess.Popen(
        ["bowtie2", "-x", bowtie_index_prefix, "-U", fq],
        stdout=subprocess.PIPE
    ) as bowtie_proc, subprocess.Popen(
        ["samtools", "view", "-Sb", "-"],
        stdin=bowtie_proc.stdout,
        stdout=open(bam_unsorted, "wb")
    ) as samtools_proc:
        bowtie_proc.stdout.close()
        samtools_proc.communicate()

    # Sort the BAM file
    subprocess.run(["samtools", "sort", "-o", bam_sorted, bam_unsorted], check=True)

    # Index the sorted BAM file
    subprocess.run(["samtools", "index", bam_sorted], check=True)

    # Clean up unsorted BAM
    os.remove(bam_unsorted)

    print(f"Done: {bam_sorted} and {bam_sorted}.bai generated.")

print("\nAll FASTQ files processed.")

### Get ribosomal coverage from BAM files

In [ ]:
# Directory containing your BAM files
bam_dir = "./"  # Change this if your BAMs are in a subfolder
# Output directory for coverage files
output_dir = "samtools_depth"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Loop through all BAM files
for bam in os.listdir():
    if bam.endswith(".bam"):
        base = os.path.splitext(bam)[0]
        output_file = os.path.join(output_dir, f"{base}_depth.txt")

        cmd = ["samtools", "depth", "-a", bam]
        print(f"Running: {' '.join(cmd)}")

        with open(output_file, "w") as f:
            subprocess.run(cmd, stdout=f, check=True)

### Append base to depth file

In [ ]:
# Step 1: Load all reference sequences into a dictionary
fasta_path = "ecoli_rRNA_consensus.fasta"
reference_seqs = {
    record.id: str(record.seq)
    for record in SeqIO.parse(fasta_path, "fasta")
}

# Step 2: Set up output directory
input_dir = "samtools_depth"
output_dir = "depth_with_bases"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
reference_seqs

In [ ]:
import pandas as pd

# Step 3: Process each depth file
for fname in os.listdir(input_dir):
    if not fname.endswith(".txt"):
        continue

    fpath = os.path.join(input_dir, fname)
    df = pd.read_csv(fpath, sep="\t", names=["seq", "pos", "depth"])

    # Step 4: Append the corresponding base from the correct reference sequence
    def get_base(row):
        seq_id = row["seq"]
        pos = row["pos"]
        seq = reference_seqs.get(seq_id)
        if seq and 0 < pos <= len(seq):
            return seq[pos - 1]  # 1-based to 0-based
        else:
            return "N"  # fallback if missing or out of bounds

    df["base"] = df.apply(get_base, axis=1)

    # Step 5: Save output
    output_file = os.path.join(output_dir, fname.replace(".txt", "_with_bases.txt"))
    df.to_csv(output_file, sep="\t", index=False)

    print(f"Processed: {fname}")

### Plot coverage

In [3]:
# Set input and output directories
input_dir = "depth_with_bases"
output_dir = "coverage_plots"
os.makedirs(output_dir, exist_ok=True)

In [8]:
# Define unique colors for each base
base_colors = {
    'a': '#1f77b4',  # blue
    't': '#ff7f0e',  # orange
    'g': '#2ca02c',  # green
    'c': '#d62728',  # red
    'N': '#7f7f7f'   # gray
}

In [9]:
# Loop through all processed depth files
for fname in os.listdir(input_dir):
    if not fname.endswith("_with_bases.txt"):
        continue

    filepath = os.path.join(input_dir, fname)
    df = pd.read_csv(filepath, sep="\t")

    # Plot coverage for each sequence in the file
    for seq_id in df['seq'].unique():
        sub_df = df[df['seq'] == seq_id]

        # Start figure
        plt.figure(figsize=(14, 4))

        # Plot each base in a different color
        for base in sorted(sub_df['base'].unique()):
            base_df = sub_df[sub_df['base'] == base]
            plt.scatter(base_df['pos'], base_df['depth'], s=1, color=base_colors.get(base, '#000000'), label=base)

        # Titles and labels
        plt.title(f"{fname.replace('_with_bases.txt','')} – {seq_id} Coverage")
        plt.xlabel("Position")
        plt.ylabel("Read Depth")
        plt.legend(title="Base", markerscale=5)
        plt.tight_layout()

        # Save PDF
        pdf_name = f"{fname.replace('_with_bases.txt','')}_{seq_id}_coverage.pdf"
        pdf_path = os.path.join(output_dir, pdf_name)
        plt.savefig(pdf_path)
        plt.close()

        print(f"Saved: {pdf_path}")


Saved: coverage_plots/SRR11584024.sorted_depth_16S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584024.sorted_depth_23S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584024.sorted_depth_5S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584027.sorted_depth_16S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584027.sorted_depth_23S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584027.sorted_depth_5S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584026.sorted_depth_16S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584026.sorted_depth_23S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584026.sorted_depth_5S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584025.sorted_depth_16S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584025.sorted_depth_23S_consensus_coverage.pdf
Saved: coverage_plots/SRR11584025.sorted_depth_5S_consensus_coverage.pdf
Saved: coverage_plots/SRR10533476.sorted_depth_16S_consensus_coverage.pdf
Saved: coverage_plots/SRR10533476.sorted_d

### Combined coverage plot

In [19]:
# Parameters
input_dir = "depth_with_bases"
output_dir = "combined_coverage_boxplots_chunked"
chunk_size = 300  # number of positions per plot
os.makedirs(output_dir, exist_ok=True)


In [20]:

# Define colors per base (case-insensitive)
base_colors = {
    'a': '#1f77b4',  # blue
    't': '#ff7f0e',  # orange
    'g': '#2ca02c',  # green
    'c': '#d62728',  # red
    'n': '#7f7f7f'   # gray
}


In [21]:
# Load all *_with_bases.txt files
combined_data = []
for fname in os.listdir(input_dir):
    if fname.endswith("_with_bases.txt"):
        sample = fname.replace("_with_bases.txt", "")
        fpath = os.path.join(input_dir, fname)
        df = pd.read_csv(fpath, sep="\t")
        df["sample"] = sample
        combined_data.append(df)

# Combine into one DataFrame
all_df = pd.concat(combined_data, ignore_index=True)
all_df.head()

,seq,pos,depth,base,sample
0,16S_consensus,1,3468,a,SRR11584024.sorted_depth
1,16S_consensus,2,4360,a,SRR11584024.sorted_depth
2,16S_consensus,3,7524,a,SRR11584024.sorted_depth
3,16S_consensus,4,7719,t,SRR11584024.sorted_depth
4,16S_consensus,5,7768,t,SRR11584024.sorted_depth


In [26]:
# Generate plots for each rRNA sequence in chunks
for seq_id in all_df["seq"].unique():
    seq_df = all_df[all_df["seq"] == seq_id]

    # Group by position: get list of depths and base
    grouped = seq_df.groupby("pos").agg({
        "depth": lambda x: list(x),
        "base": lambda x: x.iloc[0].lower()  # assume base is same across samples
    }).reset_index()

    max_pos = grouped["pos"].max()

    # Split into chunks
    for start in range(1, max_pos + 1, chunk_size):
        end = min(start + chunk_size - 1, max_pos)
        chunk = grouped[(grouped["pos"] >= start) & (grouped["pos"] <= end)]

        if chunk.empty:
            continue

        # Plot
        plt.figure(figsize=(15, 5))
        box = plt.boxplot(chunk["depth"], positions=chunk["pos"], patch_artist=True, showfliers=False)

        for patch, base in zip(box['boxes'], chunk["base"]):
            patch.set_facecolor(base_colors.get(base, '#000000'))

        plt.title(f"{seq_id} Coverage – Positions {start}-{end}")
        plt.xlabel("Position")
        plt.ylabel("Read Depth")
        plt.xticks(rotation=45,fontsize=2)
        plt.tight_layout()

        # Save plot
        chunk_file = os.path.join(output_dir, f"{seq_id}_boxplot_{start}_{end}.pdf")
        plt.savefig(chunk_file)
        plt.close()

        print(f"Saved: {chunk_file}")

Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_1_300.pdf
Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_301_600.pdf
Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_601_900.pdf
Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_901_1200.pdf
Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_1201_1500.pdf
Saved: combined_coverage_boxplots_chunked/16S_consensus_boxplot_1501_1542.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_1_300.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_301_600.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_601_900.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_901_1200.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_1201_1500.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_1501_1800.pdf
Saved: combined_coverage_boxplots_chunked/23S_consensus_boxplot_1801_2100.pdf
Save